In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%run ../script/config.py

In [11]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt']

In [12]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f.txt',
 '../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt',
 '../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt',
 '../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [13]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [14]:
%run ../script/webnlg.py

In [15]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())



def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    return pd.DataFrame({'models': models_paths,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [16]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [17]:
e = test_with_lex.sample()

df = print_entry_and_models(e)

Triple info: {'category': 'WrittenWork', 'eid': 'Id405', 'idx': '0_404', 'ntriples': 3}

	Modified triples:

1634:_The_Bavarian_Crisis | numberOfPages | "448"
1634:_The_Bavarian_Crisis | author | "Virginia DeMarce and Eric Flint"
1634:_The_Bavarian_Crisis | mediaType | "Print"


	Lexicalizations:

Available in print form, '1634: The Bavarian Crisis' by Virginia DeMarce and Eric Flint has 448 pages.
1634: The Bavarian Crisis is written by Virginia DeMarce and Eric Flint and has 448 pages in print.
1634: The Bavarian Crisis was written by Virginia DeMarce and Eric Flint, has 448 pages and is available in print form.




In [18]:
df

,bleu,models,texts
8,0.723776,../data/webnlg2017/submissions/tilburg/smt_test.out.ordered,: the bavarian crisis has 448 pages and 1634 : the bavarian crisis is written by virginia demarce and eric flint and is available in print form .
4,0.637202,../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt,"1634 : the bavarian crisis , written by virginia demarce and eric flint , is available in print form and has 448 pages ."
5,0.583027,../data/webnlg2017/submissions/melbourne/final_result.txt,the print book 1634: the bavarian crisis has 448 pages and was written by virginia demarce and eric flint .
9,0.582823,../data/webnlg2017/submissions/tilburg/template_test.out.ordered,1634 : the bavarian crisis by virginia demarce and eric flint is 448 pages long and in print .
12,0.579558,../data/webnlg2017/submissions/baseline_sorted.txt,"1634 : the bavarian crisis , written by virginia demarce and eric flint , is in print and has 448 pages ."
6,0.572685,../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt,"1634 : the bavarian crisis , written by virginia demarce and eric flint , is available in hardcover and has 448 pages ."
3,0.548802,../data/models/5 - Model - Template Based - roots_b6a7d881636c7be572e04d2d10a4523354324356.txt,1634: The Bavarian Crisis has 448 pages. 1634: The Bavarian Crisis was written by Virginia DeMarce and Eric Flint. 1634: The Bavarian Crisis is available in Print.
10,0.529793,../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt,1634 : the bavarian crisis by virginia demarce and eric flint was produced in print and is 448 pages long .
2,0.439620,../data/models/5 - Model - Template Based - roots_6dd871aa81bea6fcaf35b71faf932af2e82e17e6.txt,1634: The Bavarian Crisis has 448 pages. 1634: The Bavarian Crisis was written by Virginia DeMarce and Eric Flint. 1634: The Bavarian Crisis was published in Print.
1,0.298857,../data/models/5 - Model - Template Based - roots_604049f5126c8e7723be2bcc42a5b3ba96fb29e8.txt,1634: The Bavarian Crisis has 448 pages. 1634: The Bavarian Crisis was written by Virginia De Marce and Eric Flint. 1634: The Bavarian Crisis was published in Print.


In [29]:
%run ../script/data_alignment.py

import spacy

nlp = spacy.load('en_core_web_lg')

In [81]:
%run ../script/data_alignment.py

In [82]:
da = RootDataAlignmentModel(similarity_metric=similarity.token_sort_ratio)

In [83]:
first_lexicalization = e.ldf.ltext.values.tolist()[1]
# uses the first triple
first_triple = e.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]

da.render_aligned(nlp(first_lexicalization), first_triple)